In [ ]:
%cd ..

In [ ]:
import json

In [ ]:
file = open("../../../out_60deg.json")

In [ ]:
# returns JSON object as 
# a dictionary
data = json.load(file)

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

from matplotlib import pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")

In [ ]:
x_data = []
y_data = []

#edep_data = np.zeros((80,80), dtype=float)

for eventid in data["event"]:
    edep_data = np.zeros((80,80), dtype=float)
    index = np.array([0,0])
    # print(eventid)
    if "tileHitEdep" not in data["event"][str(eventid)].keys():
        #print(eventid)
        continue
    if data["event"][str(eventid)]["edep_detector"] <= 10:
        continue
    for key in data["event"][str(eventid)]["tileHitEdep"]:
        #x_data.append(data["geometry"]["tiles"][key]["posX"])
        #y_data.append(data["geometry"]["tiles"][key]["posY"])
        x = (int(key))%80
        y = (int(key)) // 80
        edep = data["event"][str(eventid)]["tileHitEdep"][key]
        edep_data[x][y] = np.log(edep)
        index[0] += x*edep
        index[1] += y*edep        

    #index = np.round(index/data["event"][str(eventid)]["edep_detector"])
    #print(index)
    #index = unravel_index(edep_data.argmax(), edep_data.shape)
    #print(index1 , " | ", index)
    #edep_data_test = edep_data[int(index[0])-20:int(index[0])+20,int(index[1])-20:int(index[1])+20]
    edep_data_test = edep_data[:,20:60]
    if (np.sum(np.exp(edep_data_test))/data["event"][str(eventid)]["edep_detector"]) < 0.8:
        print("WARNING")
        continue
    
    if edep_data_test.shape != (80,40):
        print("ERROR")
        continue
    x_data.append(np.array(edep_data_test))
    y_data.append(data["event"][str(eventid)]["particle_energy"]/5000)

In [ ]:
x_data = []
y_data = []

#edep_data = np.zeros((80,80), dtype=float)

for eventid in data["event"]:
    edep_data = np.zeros((80,80), dtype=float)
    # print(eventid)
    if "tileHitEdep" not in data["event"][str(eventid)].keys():
        #print(eventid)
        continue
    if data["event"][str(eventid)]["edep_detector"] <= 10:
        continue
    for key in data["event"][str(eventid)]["tileHitEdep"]:
        #x_data.append(data["geometry"]["tiles"][key]["posX"])
        #y_data.append(data["geometry"]["tiles"][key]["posY"])
        x = (int(key)) % 80
        y = (int(key)) // 80
        edep = data["event"][str(eventid)]["tileHitEdep"][key]
        edep_data[x][y] = edep
   

    x_data.append(np.array(edep_data)/100)
    y_data.append(data["event"][str(eventid)]["particle_energy"]/5000)

In [ ]:
len(x_data)

In [ ]:
np.array(x_data).shape

In [ ]:
x_train = torch.Tensor(np.array(x_data).reshape(-1, 1, 80, 40).astype('float32'))

y_train = torch.Tensor(y_data).unsqueeze(-1)

In [ ]:
# Test if everything is still ok
plt.imshow(np.array(x_train[0][0]), cmap="magma_r")
plt.show()

In [ ]:
class cnn_data( Dataset ):
    
    def __init__( self, imgs, labels ):
        self.imgs = imgs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.imgs[idx], self.labels[idx]

In [ ]:
train_cnn_dataset = cnn_data(x_train, y_train.float())

In [ ]:
train_cnn_dataloader = DataLoader( train_cnn_dataset, batch_size=32, shuffle=True )

In [ ]:
class ConvNet2D(nn.Module):
    
    def __init__(self, in_ch=1, ch=4, out_dim=1, img_sz=40):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, ch, kernel_size=4, bias=True, stride=1, padding=2)
        self.conv2 = nn.Conv2d(ch, 1, kernel_size=4, bias=True, stride=1, padding=2)
        self.max = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(int(80*40/16), 200, bias=True)
        self.dense2 = nn.Linear(200, 100, bias=True)
        self.dense3 = nn.Linear(100, out_dim, bias=True)
        
    def forward(self, x):
        self.net = nn.Sequential(
            #self.max,
            self.conv1,
            nn.ReLU(),
            self.max,
            nn.ReLU(),
            self.conv2,
            self.max,
            self.flatten,
            self.dense1,
            nn.ReLU(),
            self.dense2,
            nn.ReLU(),
            self.dense3,
            #nn.ReLU(),
            #self.dense4,
            #nn.ReLU()
            #nn.Sigmoid()
        )
        y = self.net(x)
        #return torch.exp(y)
        return y
    

In [ ]:
class ConvNet2D(nn.Module):
    
    def __init__(self, in_ch=1, ch=4, out_dim=1, img_sz=40):
        super().__init__()
        self.flatten = nn.Flatten()
        self.denseIN = nn.Linear(img_sz**2, 1, bias=True)
        self.dense1 = nn.Linear(400, 200, bias=True)
        self.dense2 = nn.Linear(200, 100, bias=True)
        self.dense3 = nn.Linear(100, out_dim, bias=True)
        
    def forward(self, x):
        self.net = nn.Sequential(
            #self.max,
            self.flatten,
            self.denseIN,
            nn.ReLU(),
            #self.dense1,
            #nn.ReLU(),
            #self.dense2,
            #nn.ReLU(),
            #self.dense3,
            #nn.ReLU(),
            #self.dense4,
            #nn.ReLU()
            #nn.Sigmoid()
        )
        y = self.net(x)
        #return torch.exp(y)
        return y
    

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
def train_epoch( dataloader, model, loss_fn, optimizer ):
    
    size = len( dataloader.dataset )
    
    for batch, (X, y) in enumerate(dataloader):

        # pass data through network
        pred = model(X)
        
        # compute loss
        loss = loss_fn( pred, y )

        # reset gradients in optimizer
        optimizer.zero_grad()
        
        # compute gradients
        loss.backward()
        
        # update weights with optimizer
        optimizer.step()
        
        # print the training loss every 100 updates
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print( f"current batch loss: {loss:>7f}  [{current:>5d}/{size:>5d}]" )

In [ ]:
# a useful function to present things clearer
def separator():
    print( "-----------------------------------------------" )

epochs = 20

# re-initialise the model and the optimizer
model = ConvNet2D(ch=10, img_sz=80).to(device)
learning_rate = 5e-4
weight_decay = 1e-1
optimizer = torch.optim.Adam( model.parameters(), lr=learning_rate, weight_decay=weight_decay )
separator()
print( "model architecture ")
separator()
print( model )

# track train and val losses
trn_losses = []
val_losses = []

for t in range(epochs):
    separator()
    print( f"Epoch {t+1}" )
    separator()
    train_epoch( train_cnn_dataloader, model, loss_fn, optimizer )
    separator()
    #trn_loss = trn_pass( train_cnn_dataloader, model, loss_fn )
    #trn_losses.append( trn_loss )
    #separator()
    #val_loss = val_pass( val_cnn_dataloader, model, loss_fn )
    #val_losses.append( val_loss )
    separator()
    print( "|" )
    
print("Done!")

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():
    #model.eval()
    #model_ev = model.eval()
    pred = model(train_cnn_dataset[1][0])
    print(pred)

In [ ]:
float(pred)

In [ ]:
float(train_cnn_dataset[1][1])

In [ ]:
plt.imshow(np.array(x_train.mean(0).reshape((80,80))), cmap="magma_r")
plt.show()

In [ ]:
abw = []
with torch.no_grad():
    for index in range(47000):
        pred = model(train_cnn_dataset[index][0])
        abw.append(float(pred)-float(train_cnn_dataset[index][1]))

In [ ]:
abw_np = np.array(abw)*5000

In [ ]:
abw_np.mean()

In [ ]:
plt.hist(abw_np, bins=100)
#plt.xlim(0,5000)
plt.show()

In [ ]:
np.std(abw_np)